In [1]:
import sys

import numpy as np

sys.path.append('/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings/src')

# Azure Data Lake libraries
import azure_data_lake_interface as adl

# Data analysis libraries
import pandas as pd

# Helper function libraries
import helper_functions as hf

In [2]:
# attach to the data lake
config = hf.load_config("config/datalake_config.json", flush_cache=True)
service_client = adl.get_azure_service_client(config["blob_url"])
file_system_client = adl.get_azure_file_system_client(service_client, "consolidated")

In [5]:
# get data
transactions, line_items = adl.get_transactions_and_line_items(file_system_client, "CustInvc", data_state="enhanced")
save_line_items = line_items.copy()

In [27]:
high_prices = line_items[~line_items.highest_recent_price.isna()].copy()
high_prices["difference"] = high_prices.quote_po_rate - high_prices.highest_recent_price
high_prices["pct_diff"] = round(high_prices.difference/high_prices.quote_po_rate * 100, 2)

In [28]:
quote_matches = high_prices[abs(high_prices.difference) < 1]
quote_higher = high_prices[high_prices.difference >= 1]
quote_lower = high_prices[high_prices.difference <= -1]

In [29]:
print(f"quote matches: {(quote_matches.shape[0]/high_prices.shape[0])*100:.2f}%")
print(f"quote below: {(quote_lower.shape[0]/high_prices.shape[0])*100:.2f}%")
print(f"quote above: {(quote_higher.shape[0]/high_prices.shape[0])*100:.2f}%")

quote matches: 58.29%
quote below: 35.65%
quote above: 6.07%


In [31]:
quote_lower[['difference', 'pct_diff']].describe()

/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings/Phase 2/Operational Visibility Workstream/MarginAnalysis/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,difference,pct_diff
count,87071.000000,87071.00
mean,-397.050655,-inf
std,3738.450575,NaN
min,-347628.740000,-inf
25%,-139.950000,-65.63
50%,-34.670000,-25.67
75%,-7.160000,-8.17
max,-1.000000,-0.00


### Sales by Commission Type, Jan-2023 to Apr-2025

In [4]:
line_items['Month'] = line_items.created_date.dt.strftime('%Y-%m')

# Group by Month and commission_or_mfr_direct to get total sales
sales_by_month = line_items.groupby(['Month', 'commission_or_mfr_direct'])['total_amount'].sum().reset_index()

# Pivot the data to show commission vs non-commission sales side by side
sales_pivot = sales_by_month.pivot(index='Month',
                                   columns='commission_or_mfr_direct',
                                   values='total_amount')
sales_pivot.fillna(0, inplace=True)

# Rename columns for clarity
sales_pivot.columns = ['Non-Commission Sales', 'Commission Sales']

# Add total sales column
sales_pivot['Total Sales'] = sales_pivot.sum(axis=1)

print("Monthly Sales by Commission Type:")
print(sales_pivot.to_string(float_format=lambda x: f'${x:,.2f}'))

Monthly Sales by Commission Type:
         Non-Commission Sales  Commission Sales    Total Sales
Month                                                         
2022-01         $6,135,490.58             $0.00  $6,135,490.58
2022-02         $6,690,763.83             $0.00  $6,690,763.83
2022-03         $9,466,324.30             $0.00  $9,466,324.30
2022-04         $9,827,403.56             $0.00  $9,827,403.56
2022-05         $9,799,709.55             $0.00  $9,799,709.55
2022-06        $13,786,530.83           $564.42 $13,787,095.25
2022-07        $12,170,738.34             $0.00 $12,170,738.34
2022-08        $14,751,701.22         $3,526.50 $14,755,227.72
2022-09        $13,599,400.24         $5,928.69 $13,605,328.93
2022-10        $19,878,089.09        $35,091.10 $19,913,180.19
2022-11        $14,384,654.66       $126,461.20 $14,511,115.86
2022-12        $12,313,286.82        $42,081.05 $12,355,367.87
2023-01        $12,110,953.00         $6,807.42 $12,117,760.42
2023-02        $12,57

In [5]:
print(f"${sales_pivot['Non-Commission Sales'].sum():,.2f}",
      f"${sales_pivot['Commission Sales'].sum():,.2f}",
      f"${sales_pivot['Total Sales'].sum():,.2f}")

$634,770,810.60 $33,044,349.62 $667,815,160.22


### Item Type Distribution for Non-Commission Line Items

In [6]:
comm_lines = line_items.query('commission_or_mfr_direct == True')
comm_lines.item_type.value_counts()

item_type
Inventory Item                7472
Non-inventory Item            1918
Service                         32
Assembly/Bill of Materials       3
Name: count, dtype: int64

In [7]:
comm_lines.query('item_type == "Inventory Item"')

,amount,assembly_component,cost_estimate_type,created_from,custcol_ava_taxamount,custcol_sa_quote_po_rate,custom_manufacturer,customer_id,description,display_name,...,level_3_category,level_4_category,level_5_category,level_6_category,vsi_mfr,vsi_item_category,total_cost,gross_profit,gross_profit_percent,Month
49819,3526.50,F,CUSTOM,Sales Order #305726,0.00,0.0,CDC,689,"SANITRX 2""",Gas/Liquid Rupture Disc,...,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,2548.80,977.70,0.277244,2022-08
52731,1424.12,F,CUSTOM,Sales Order #302486,0.00,0.0,McCrometer,290878,"MCCROMETER 3"" WAFER V-CONE DP FLOWMETER.\r\nP/...",null,...,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,1210.50,213.62,0.150001,2022-09
52732,1145.29,F,CUSTOM,Sales Order #302486,0.00,0.0,McCrometer,290878,"MCCROMETER 2"" WAFER V-CONE DP FLOWMETER \r\n...",null,...,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,973.50,171.79,0.149997,2022-09
52733,2881.28,F,CUSTOM,Sales Order #302486,0.00,0.0,Yokogawa,290878,YOKOGAWA EJA110E DP TRANS - REMOTE MOUNT * HI...,null,...,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,2701.20,180.08,0.062500,2022-09
52734,478.00,F,PURCHPRICE,Sales Order #302486,0.00,0.0,Yokogawa,290878,C13ST-3TSA0-S4-NNNNN-NNNN\r\n\r\nYokogawa 3-Va...,Yokogawa 3-Valve Manifold (Stocked Model),...,Manifold,Not Specified,New,3 valve,Not Specified,Not Specified,358.50,119.50,0.250000,2022-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393356,3670.00,F,PURCHORDERRATE,Sales Order #437372,0.00,0.0,MSA,156220,A-X5000-0-F-0-1-0-16-00-0\r\n\r\nA-X5000: MSA ...,MSA X5000 Gas Monitor - Oxygen,...,Fixed Gas,X5000,New,Not Specified,Not Specified,Not Specified,3670.00,0.00,0.000000,2025-04
393370,11420.00,F,PURCHORDERRATE,Sales Order #435887,0.00,0.0,MSA,412873,A-CG5000-0-4-1-0\r\nMSA CHILLGARD 5000\r\n4 CH...,"Deplete inventory, Do not reorder",...,Controllers,Chillagard,New,Not Specified,Not Specified,Not Specified,11420.00,0.00,0.000000,2025-04
393391,2133.05,F,PURCHORDERRATE,Sales Order #423255,0.00,0.0,Yokogawa,15882,FX1002-7-2-L/C7/M1/P1/USB1/S1\r\n \r\nFX1002: ...,Yokogawa Paperless Recorder,...,Data Recorder,FX,New,Not Specified,Not Specified,Not Specified,2133.05,0.00,0.000000,2025-04
393433,4800.00,F,PURCHORDERRATE,Sales Order #437127,0.00,0.0,Yokogawa,32942,UT35A-L00-11-00\r\nUT35A: Yokogawa Digital Ind...,Yokogawa Digital Indicating Limit Controller,...,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,Not Specified,4800.00,0.00,0.000000,2025-04
